<a href="https://colab.research.google.com/github/rishurnjan/fake_news_classifier-NLP-/blob/main/fake_news_classifier(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ** FAKE NEWS CLASSIFIER USING LSTM AND WORD EMBEDDING **

Dataset: https://www.kaggle.com/c/fake-news/data#

In [41]:
!ls -l /content/train.csv.zip  # Check if the file exists and its size
!file /content/train.csv.zip   # Check the file type

-rw-r--r-- 1 root root 38841253 Jul  7 18:56 /content/train.csv.zip
/content/train.csv.zip: Zip archive data, at least v4.5 to extract, compression method=deflate


In [42]:
import zipfile

with zipfile.ZipFile('/content/train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('fake-news')


In [43]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/fake-news/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [44]:
df=df.dropna()

In [46]:
messages=df.copy()


In [47]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
messages.reset_index(inplace=True)

In [51]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]
for i in range(len(messages)):
  rev=messages['title'][i]
  if pd.isnull(rev):
    rev=''
  rev=rev.lower()
  rev=re.sub('[^a-zA-Z]',' ',rev)
  rev=rev.split()
  rev=[PorterStemmer().stem(word) for word in rev if word not in set(stopwords.words('english'))]
  rev=' '.join(rev)
  corpus.append(rev)

In [53]:
print(corpus[0], df['title'][0])

hous dem aid even see comey letter jason chaffetz tweet House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It


In [55]:
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
one_hot_rep=[one_hot(word,5000)for word in corpus]
one_hot_rep[0]

[4853, 4209, 2967, 3417, 548, 507, 2365, 4975, 700, 1015]

In [56]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_len=20
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_len)

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [58]:
embedding_vector_feature=40
model=Sequential()
model.add(Embedding(5000,embedding_vector_feature,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [59]:
X=np.array(embedded_docs)
y=np.array(df['label'])

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [62]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 54ms/step - loss: 0.3506 - accuracy: 0.8318 - val_loss: 0.2071 - val_accuracy: 0.9117
Epoch 2/10
192/192 [==============================] - 8s 39ms/step - loss: 0.1446 - accuracy: 0.9426 - val_loss: 0.1942 - val_accuracy: 0.9148
Epoch 3/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0974 - accuracy: 0.9633 - val_loss: 0.2490 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0578 - accuracy: 0.9804 - val_loss: 0.2782 - val_accuracy: 0.9180
Epoch 5/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0341 - accuracy: 0.9902 - val_loss: 0.3306 - val_accuracy: 0.9120
Epoch 6/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0214 - accuracy: 0.9938 - val_loss: 0.4426 - val_accuracy: 0.9137
Epoch 7/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0133 - accuracy: 0.9965 - val_loss: 0.4184 - val_accuracy: 0.9

In [63]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 7ms/step


In [66]:
from sklearn.metrics import confusion_matrix
import numpy as np

y_pred_labels = np.where(y_pred > 0.5, 1, 0)
confusion_matrix(y_test, y_pred_labels)

array([[3088,  331],
       [ 211, 2405]])

In [69]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_labels))

0.9101905550952776


In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_labels))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

